In [1]:
"""
Pattern Lab Project - Team Kingsmen

Name: Mohammed Jawwadul Islam     	
ID: 011 181 182
UIU Official Email: mislam181182@bscse.uiu.ac.bd

Name: MD Fahad Al Rafi                                  
ID: 011 181 201
UIU Official Email: mrafi181201@bscse.uiu.ac.bd 

Name: Pranto Podder                                  	  
ID: 011 181 202
UIU Official Email: ppodder181202@bscse.uiu.ac.bd
"""

'\nPattern Lab Project - Team Kingsmen\n\nName: Mohammed Jawwadul Islam     \t\nID: 011 181 182\nUIU Official Email: mislam181182@bscse.uiu.ac.bd\n\nName: MD Fahad Al Rafi                                  \nID: 011 181 201\nUIU Official Email: mrafi181201@bscse.uiu.ac.bd \n\nName: Pranto Podder                                  \t  \nID: 011 181 202\nUIU Official Email: ppodder181202@bscse.uiu.ac.bd\n'

## Importing the Libraies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from collections import Counter

## Reading the dataset

In [ ]:
data = pd.read_csv('healthcare-dataset-stroke-data.csv', na_values='N/A')

In [ ]:
# Removing " " empty space between feature values  

from pandas.api.types import is_string_dtype

for column in data.columns:
    if (is_string_dtype(data[column].dtype)):
        data[column] = data[column].str.strip()

In [ ]:
X = data.loc[:, data.columns != 'stroke']
y = data['stroke']

In [ ]:
print(X.shape, y.shape)

(5110, 11) (5110,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Train - 80% , Test - 20%

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4088, 11) (1022, 11) (4088,) (1022,)


## Data Preprocessing

### Handling Missing Values

#### Simple Imputation by Sklearn - Mean/Median/Most Frequent

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
si_X_train = pd.DataFrame() # create a new dataframe to save the train dataset
si_X_test = pd.DataFrame() # create a new dataframe to save the test dataset

In [ ]:
for column in X_train.columns:
  if (is_string_dtype(X_train[column].dtype)):
    si = SimpleImputer(strategy='most_frequent')
  else:
    si = SimpleImputer(strategy='median')
  si.fit(X_train[[column]])
  si_X_train[column] = si.transform(X_train[[column]]).flatten() # Flatten 2D matrix to 1D 
  si_X_test[column] = si.transform(X_test[[column]]).flatten()

In [ ]:
si_X_train

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,28932.0,Female,36.0,0.0,0.0,Yes,Private,Rural,67.29,36.7,formerly smoked
1,37150.0,Female,34.0,0.0,0.0,Yes,Private,Rural,83.53,48.5,formerly smoked
2,71669.0,Male,60.0,0.0,0.0,Yes,Private,Rural,65.16,30.8,never smoked
3,27153.0,Female,75.0,0.0,0.0,Yes,Self-employed,Rural,78.80,29.3,formerly smoked
4,58235.0,Male,76.0,0.0,0.0,Yes,Private,Urban,58.65,25.6,smokes
...,...,...,...,...,...,...,...,...,...,...,...
4083,30457.0,Female,53.0,1.0,0.0,Yes,Govt_job,Rural,98.61,38.8,smokes
4084,24836.0,Female,61.0,0.0,0.0,Yes,Private,Rural,72.01,26.0,formerly smoked
4085,17079.0,Male,44.0,0.0,0.0,Yes,Private,Rural,94.71,28.4,smokes
4086,72340.0,Male,21.0,0.0,0.0,No,Private,Urban,120.94,29.7,formerly smoked


### Handling Text Features

#### Label Encoder

In [ ]:
categorical_features = []
for col in data.columns:
  if col=='Class':
    continue
  if is_string_dtype(data[col].dtype):
    categorical_features.append(col)

In [ ]:
categorical_features

['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
# Convert the Label Class 

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
l_X_train = pd.DataFrame() # Train dataset --> before scaling
l_X_test = pd.DataFrame() # Test dataset --> before scaling

In [ ]:
# Convert the text features

for column in X_train.columns:
  if column in categorical_features:
    l_X_train[column] = le.fit_transform(si_X_train[column])
    l_X_test[column] = le.transform(si_X_test[column])
  else:
    l_X_train[column] = si_X_train[column].copy()
    l_X_test[column] = si_X_test[column].copy()

In [ ]:
l_X_train.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64

## Oversampling the dataset

whichever classess has less number of values, we will impute and increase the values in those classes

In [ ]:
from imblearn.over_sampling import RandomOverSampler

os=RandomOverSampler(0.75) # 75%
l_X_train_ns,y_train_ns = os.fit_resample(l_X_train,y_train)

print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({0: 3893, 1: 195})
The number of classes after fit Counter({0: 3893, 1: 2919})


## Feature Scaling

#### Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

l_X_train_ns = ss.fit_transform(l_X_train_ns)
l_X_test = ss.transform(l_X_test)

In [ ]:
print(l_X_train)

           id  gender   age  ...  avg_glucose_level   bmi  smoking_status
0     28932.0       0  36.0  ...              67.29  36.7               1
1     37150.0       0  34.0  ...              83.53  48.5               1
2     71669.0       1  60.0  ...              65.16  30.8               2
3     27153.0       0  75.0  ...              78.80  29.3               1
4     58235.0       1  76.0  ...              58.65  25.6               3
...       ...     ...   ...  ...                ...   ...             ...
4083  30457.0       0  53.0  ...              98.61  38.8               3
4084  24836.0       0  61.0  ...              72.01  26.0               1
4085  17079.0       1  44.0  ...              94.71  28.4               3
4086  72340.0       1  21.0  ...             120.94  29.7               1
4087  52242.0       0  58.0  ...              59.52  33.2               2

[4088 rows x 11 columns]


## Model Building

## Classification Evaluatin Metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

def evaluate_preds(y_test,y_pred):
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred) 
    f1 = f1_score(y_test,y_pred)
    mcc = matthews_corrcoef(y_test,y_pred)

    metric_dict = {
        "accuracy":round(accuracy,2),
        "precision":round(precision,2),
        "recall":round(recall,2),
        "f1":round(f1,2),
        "mcc": mcc 
    } # A dictionary that stores the results of the evaluation metrics
    
    print(f"Acc: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1:.2f}")
    print(f'MCC Score: {mcc:.2f}')
    
    return metric_dict

### SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf',random_state=0)
svc.fit(l_X_train_ns,y_train_ns)

SVC(random_state=0)

In [ ]:
y_pred = svc.predict(l_X_test)
model_metrics = evaluate_preds(y_test, y_pred)

Acc: 82.29%
Precision: 0.14
Recall: 0.46
F1 score: 0.22
MCC Score: 0.18


### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(l_X_train_ns,y_train_ns)

GaussianNB()

In [ ]:
y_pred = naive.predict(l_X_test)
model_metrics = evaluate_preds(y_test, y_pred)

Acc: 79.45%
Precision: 0.16
Recall: 0.70
F1 score: 0.27
MCC Score: 0.27


### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.fit(l_X_train_ns,y_train_ns)

LogisticRegression()

In [ ]:
y_pred = logistic.predict(l_X_test)
model_metrics = evaluate_preds(y_test, y_pred)

Acc: 81.02%
Precision: 0.17
Recall: 0.65
F1 score: 0.27
MCC Score: 0.26


### k Nearest Neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=40)
neigh.fit(l_X_train_ns,y_train_ns)

KNeighborsClassifier(n_neighbors=40)

In [ ]:
y_pred = neigh.predict(l_X_test)
model_metrics = evaluate_preds(y_test, y_pred)

Acc: 76.32%
Precision: 0.13
Recall: 0.59
F1 score: 0.21
MCC Score: 0.19


### RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, criterion='entropy')
rf.fit(l_X_train_ns,y_train_ns)

RandomForestClassifier(criterion='entropy', n_estimators=500, n_jobs=-1)

In [ ]:
y_pred = rf.predict(l_X_test)
model_metrics = evaluate_preds(y_test, y_pred)

Acc: 94.81%
Precision: 1.00
Recall: 0.02
F1 score: 0.04
MCC Score: 0.13


## Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

### SVM

In [ ]:
# RandomizedSearchCV will refit the estimator, i.e. model object using the best found parameters on the whole dataset.

parameters = [ # list of different combination of hyperparameters that we want to test
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
]
# also, two dictionaries --> for testing 2 different kernels. {key - parameters(name must match documentation), value - range of values to try}

random_cv = RandomizedSearchCV(estimator = svc,
                           param_distributions = parameters,
                           scoring = 'accuracy', # evaluation metric. 'accuracy' for classification here
                           cv = 5, # k = 5, no. of k-train test folds
                           verbose=2, # print out the results
                           n_jobs = -1, # set processor of machine. -1 --> means all processor will be in  use 
                           refit=True) 

# RandomizedSearchCV is only applied on train set
random_cv.fit(l_X_train_ns, y_train_ns)

# get best accuracy, and the set of parameters that led to that best accuracy
best_accuracy = random_cv.best_score_
best_parameters = random_cv.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Accuracy: 97.56 %
Best Parameters: {'kernel': 'rbf', 'gamma': 0.8, 'C': 1}


### Random Forest Classifier

In [ ]:
parameters = [ # list of different combination of hyperparameters that we want to test
              {'n_estimators': range(10,300,10), 'criterion': ['gini', 'entropy'], 'max_features':['auto','sqrt','log2']}
]


random_cv = RandomizedSearchCV(estimator = rf,
                           param_distributions = parameters,
                           scoring = 'accuracy', 
                           cv = 5, 
                           verbose=2, 
                           n_jobs = -1, 
                           refit=True) 


random_cv.fit(l_X_train_ns, y_train_ns)


best_accuracy = random_cv.best_score_
best_parameters = random_cv.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Accuracy: 99.49 %
Best Parameters: {'n_estimators': 80, 'max_features': 'sqrt', 'criterion': 'gini'}
